In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install ultralytics

In [4]:
from ultralytics import YOLO

# Load the trained model
model = YOLO('/content/drive/MyDrive/Colab Notebooks/Stage 1/run_l/exp5/weights/best.pt')

# Evaluate the model on the test set
results = model.val(
    data='/content/drive/MyDrive/Colab Notebooks/Stage 1/dataset.yaml',  # same data used in training
    imgsz=640,
    split='test',  # explicitly run on the test set
    device='cuda'
)

# Print out key evaluation metrics
print("\n--- Evaluation Results ---")
print(f"mAP50: {results.box.map50:.4f}")
print(f"mAP50-95: {results.box.map:.4f}")
print(f"Preprocess time: {results.speed['preprocess']:.2f} ms")
print(f"Inference time: {results.speed['inference']:.2f} ms")
print(f"Postprocess time: {results.speed['postprocess']:.2f} ms")

# If you want to see everything in the results dictionary
#print("\n--- Full Results Object ---")
#print(results)

Ultralytics 8.3.110 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
YOLOv8l summary (fused): 112 layers, 43,607,379 parameters, 0 gradients, 164.8 GFLOPs
val: Fast image access ✅ (ping: 0.3±0.1 ms, read: 27.2±16.2 MB/s, size: 28.7 KB)


val: Scanning /content/drive/MyDrive/Colab Notebooks/Stage 1/labels/test.cache... 826 images, 0 backgrounds, 0 corrupt: 100%|██████████| 826/826 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 52/52 [00:06<00:00,  7.56it/s]


                   all        826        828      0.996      0.993      0.995      0.737
Speed: 0.2ms preprocess, 2.6ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs/detect/val3

--- Evaluation Results ---
mAP50: 0.9946
mAP50-95: 0.7367
Preprocess time: 0.25 ms
Inference time: 2.58 ms
Postprocess time: 0.78 ms


In [5]:
from ultralytics import YOLO

# Load the trained model
model = YOLO('/content/drive/MyDrive/Colab Notebooks/Stage 2/run_l/exp2/weights/best.pt')

# Evaluate the model on the test set
results = model.val(
    data='/content/drive/MyDrive/Colab Notebooks/Stage 2/dataset.yaml',  # same data used in training
    imgsz=640,
    split='test',  # explicitly run on the test set
    device='cuda'
)

# Print out key evaluation metrics
print("\n--- Evaluation Results ---")
print(f"mAP50: {results.box.map50:.4f}")
print(f"mAP50-95: {results.box.map:.4f}")
print(f"Preprocess time: {results.speed['preprocess']:.2f} ms")
print(f"Inference time: {results.speed['inference']:.2f} ms")
print(f"Postprocess time: {results.speed['postprocess']:.2f} ms")

# If you want to see everything in the results dictionary
#print("\n--- Full Results Object ---")
#print(results)

Ultralytics 8.3.110 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
YOLOv8l summary (fused): 112 layers, 43,629,738 parameters, 0 gradients, 164.9 GFLOPs
val: Fast image access ✅ (ping: 0.5±0.3 ms, read: 0.1±0.0 MB/s, size: 23.6 KB)


val: Scanning /content/drive/MyDrive/Colab Notebooks/Stage 2/labels/test.cache... 382 images, 2 backgrounds, 0 corrupt: 100%|██████████| 382/382 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 24/24 [00:04<00:00,  5.63it/s]


                   all        382       3100       0.98      0.961      0.985      0.769
                     0        257        389      0.994      0.969      0.978      0.658
                     1        212        277      0.993      0.989      0.994      0.798
                     2        159        203      0.988       0.98      0.991       0.77
                     3        174        222      0.995      0.991      0.993      0.743
                     4        256        371      0.986      0.995      0.989      0.788
                     5        174        232      0.996      0.981      0.995      0.785
                     6        163        219      0.995      0.968      0.995      0.767
                     7        134        182      0.989      0.965      0.988      0.765
                     8        218        293      0.993       0.99      0.993      0.795
                     9         30         32      0.986          1      0.995      0.752
                    1

In [ ]:
import cv2
import torch
from ultralytics import YOLO

def check(s):
    if len(s) == 6 and s[0].isalpha() and s[1].isalpha():
        return 0
    if "NN" in s and len(s) in {8, 9, 10}:
        return 0
    if len(s) not in {8, 9}:
        return 1
    if not s[2].isalpha():
        return 1
    if any(not c.isdigit() for i, c in enumerate(s) if i != 2 and i != 3):
        return 1
    return 0

plate_detector = YOLO("/content/drive/MyDrive/Colab Notebooks/Stage 1/run_l/exp5/weights/best.pt")
char_recognizer = YOLO("/content/drive/MyDrive/Colab Notebooks/Stage 2/run_l/exp2/weights/best.pt")

def display_plate(frame, plate_text, x1, y1, x2, y2):
    cv2.rectangle(frame, (x1, y1), (x2, y2), (16, 12, 23), 2)
    text_size = cv2.getTextSize(plate_text, cv2.FONT_HERSHEY_SIMPLEX, 0.9, 2)[0]
    cv2.rectangle(frame, (x1, y1 - text_size[1] - 10), (x1 + text_size[0], y1), (16, 12, 23), -1)
    cv2.putText(frame, plate_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (175,154,247), 2)

def recognize_plate_video(input_path, output_path):
    cap = cv2.VideoCapture(input_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, cap.get(cv2.CAP_PROP_FPS), (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))) )

    plates_dict1 = {}

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        plates_dict2 = {}

        plate_results = plate_detector(frame)

        for result in plate_results:
            for box in result.boxes.xyxy:
                x1, y1, x2, y2 = map(int, box)
                plate_region = frame[y1:y2, x1:x2]

                conf = 0
                plate_text = ''

                if plate_text == '':
                    char_dict = {}
                    char_results = char_recognizer(plate_region)

                    for char_result in char_results:
                        for idx, char_box in enumerate(char_result.boxes.xyxy):
                            cx, cy, w, h = char_box
                            cls_index = int(char_result.boxes.cls[idx])
                            conf += char_result.boxes.conf[idx].item()

                            if cls_index < 9:
                                char_label = chr(cls_index + 49)
                            elif cls_index < 17:
                                char_label = chr(cls_index + 56)
                            elif cls_index < 21:
                                char_label = chr(cls_index + 58)
                            elif cls_index == 21:
                                char_label = chr(cls_index + 59)
                            elif cls_index < 26:
                                char_label = chr(cls_index + 61)
                            elif cls_index < 29:
                                char_label = chr(cls_index + 62)
                            else:
                                char_label = '0'

                            char_dict[int(idx)] = (char_label, cx, cy, w, h)

                    char_items = sorted(char_dict.items(), key=lambda item: item[1][1])

                    i = 0
                    front = 0
                    flip = 0

                    while i < len(char_items) - 1:
                        idx1, (char1, cx1, cy1, mx1, my1) = char_items[i]
                        idx2, (char2, cx2, cy2, mx2, my2) = char_items[i + 1]

                        print(char1)
                        print(char_items)
                        print('front =', front)

                        if (my1) - (cy2) < (3 / 10) * (my1 - cy1):
                            if flip == char_items[i + 1]:
                                break
                            if flip == 0:
                                flip = char_items[i + 1]
                            char_items.append(char_items.pop(i + 1))
                            front = i+1
                            i-=1
                        elif (my2) - (cy1) < (3 / 10) * (my2 - cy2):
                            char_items.insert(front, char_items.pop(i + 1))
                            #char_items.append(char_items.pop(i+1))
                            i = front - 1
                            front += 1
                        i += 1

                    plate_text = "".join([char for _, (char, _, _, _, _) in char_items])

                    if conf != 0:
                      conf = conf/len(plate_text)

                if check(plate_text) == 0:
                    oiou = 0.4
                    for plate, (px1, py1, px2, py2, cf) in plates_dict1.items():
                        ix1, iy1 = max(px1, x1), max(py1, y1)
                        ix2, iy2 = min(px2, x2), min(py2, y2)

                        intersection_area = max(0, ix2 - ix1) * max(0, iy2 - iy1)
                        box_area = (x2 - x1) * (y2 - y1)
                        plate_area = (px2 - px1) * (py2 - py1)
                        iou = intersection_area / float(box_area + plate_area - intersection_area)

                        if iou >= oiou and cf > conf:
                            plate_text = plate
                            conf = cf
                            oiou = iou
                            break
                    plates_dict2[plate_text] = (x1, y1, x2, y2, conf)
                    display_plate(frame, plate_text, x1, y1, x2, y2)
                else:
                    oiou = 0.4
                    conf = 0
                    for plate, (px1, py1, px2, py2, cf) in plates_dict1.items():
                        ix1, iy1 = max(px1, x1), max(py1, y1)
                        ix2, iy2 = min(px2, x2), min(py2, y2)

                        intersection_area = max(0, ix2 - ix1) * max(0, iy2 - iy1)
                        box_area = (x2 - x1) * (y2 - y1)
                        plate_area = (px2 - px1) * (py2 - py1)
                        iou = intersection_area / float(box_area + plate_area - intersection_area)

                        if iou >= oiou and cf > conf:
                            plate_text = plate
                            conf = cf
                            oiou = iou
                            break

                    if check(plate_text) == 0:
                        plates_dict2[plate_text] = (x1, y1, x2, y2, conf)
                        display_plate(frame, plate_text, x1, y1, x2, y2)
                    elif plate_text != '':
                        display_plate(frame, '?', x1, y1, x2, y2)

        plates_dict1 = plates_dict2
        out.write(frame)

    cap.release()
    out.release()

# Paths
input_path = "/content/drive/MyDrive/Colab Notebooks/Test_Vid/Test1.mp4"
output_path = "/content/drive/MyDrive/Colab Notebooks/Output_Vid/Output0.mp4"

recognize_plate_video(input_path, output_path)

Streaming output truncated to the last 5000 lines.
0: 640x480 3 0s, 13.6ms
Speed: 2.8ms preprocess, 13.6ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 480)

0: 640x640 2 1s, 1 4, 1 6, 15.5ms
Speed: 3.2ms preprocess, 15.5ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)
2
[(3, ('2', tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tensor(2.1004, device='cuda:0'), tensor(15.0667, device='cuda:0'))), (0, ('2', tensor(13.6821, device='cuda:0'), tensor(16.5964, device='cuda:0'), tensor(17.5280, device='cuda:0'), tensor(22.9561, device='cuda:0'))), (2, ('7', tensor(18.1729, device='cuda:0'), tensor(16.4745, device='cuda:0'), tensor(22.6453, device='cuda:0'), tensor(23.1028, device='cuda:0'))), (1, ('5', tensor(23.5975, device='cuda:0'), tensor(16.4485, device='cuda:0'), tensor(27.2589, device='cuda:0'), tensor(23.3133, device='cuda:0')))]
front = 0
2
[(3, ('2', tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tensor(2.1004, device='cuda:0')